In [18]:
import pandas as pd
navigator = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)'
url_base = 'https://sofifa.com/player/'
from bs4 import BeautifulSoup
import requests

import pandas as pd
import streamlit as st
import streamlit_authenticator as stauth
from streamlit_option_menu import option_menu
import ast
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
from bs4 import BeautifulSoup
import pickle
import requests
import re
from bs4 import BeautifulSoup
import requests

In [19]:
df = pd.read_csv('/Users/kilian/Documents/GitHub/Projet-3/STREAMLIT/BD/players_3120.csv')

In [20]:
df = df.sample(5)

In [23]:
def end_contract(id):
    id = int(id)
    url_finale_title = f'{url_base}{id}'
    html_title = requests.get(url_finale_title, headers={'User-Agent': navigator})
    html_title2 = html_title.content
    soup_title = BeautifulSoup(html_title2, 'html.parser')

    for balise_parent in soup_title.find_all('div', class_='grid attribute'):
        if 'Contract valid until' in balise_parent.get_text().strip():
                end_contract = balise_parent.get_text().strip()

    try:
        end_year = max(re.findall('\d{4,}', end_contract))
        end_year = int(end_year)
    except:
        end_year = 'Unknown'
    
    return end_year


def overall(id):
    id = int(id)
    url_finale_title = f'{url_base}{id}'
    html_title = requests.get(url_finale_title, headers={'User-Agent': navigator})
    html_title2 = html_title.content
    soup_title = BeautifulSoup(html_title2, 'html.parser')

    for balise_parent2 in soup_title.find_all('div', class_= 'grid'):
            if 'Overall' in balise_parent2.get_text().strip():
                overall = balise_parent2.get_text().strip()

    try:
        overall = int(overall[:2])
    except:
        overall = 'Unknown'

    return overall

def salaire(id):
    id = int(id)
    url_finale_title = f'{url_base}{id}'
    html_title = requests.get(url_finale_title, headers={'User-Agent': navigator})
    html_title2 = html_title.content
    soup_title = BeautifulSoup(html_title2, 'html.parser')

    for balise_parent2 in soup_title.find_all('div', class_='col'):
        if 'Value' in balise_parent2.get_text().strip():
            valeur = balise_parent2.get_text().strip()
        if 'Wage' in balise_parent2.get_text().strip():
            salary = balise_parent2.get_text().strip()

    valeur = valeur[:-5]
    salary = salary[:-4]
                
    chiffres = [valeur, salary]
    new_chiffres = []

    for element in chiffres:
        if 'M' not in element and 'K' not in element:
            new_chiffres.append(int(element.replace('€', '')))
        elif 'K' in element and '.' not in element:
            new_chiffres.append(int(element.replace('K', '000').replace('€', '')))
        elif 'K' in element and '.'  in element:
            new_chiffres.append(int(element.replace('K', '00').replace('€', '')))
        elif 'M' in element and '.' not in element:
            new_chiffres.append(int(element.replace('M', '000000').replace('€', '')))
        elif 'M' in element and '.' in element:
            new_chiffres.append(int(element.replace('M', '00000').replace('.', '').replace('€', '')))

    valeur = int(new_chiffres[0])

    salary = int(new_chiffres[1])
    salary = str(salary)

    if len(salary) > 3:
        if len(salary) > 6:
            if len(salary) > 9:
                if len(salary) > 12:
                    salary = salary[:-12] + ' ' + salary[-12:-9] + ' ' + salary[-9:-6] + ' ' + salary[-6:-3] + ' ' + salary[-3:]
                else:
                    salary = salary[:-9] + ' ' + salary[-9:-6] + ' ' + salary[-6:-3] + ' ' + salary[-3:]
            else:
                salary = salary[:-6] + ' ' + salary[-6:-3] + ' ' + salary[-3:]
        else:
            salary = salary[:-3] + ' ' + salary[-3:]
    else:
        salary = salary

    salary = salary + ' €'

    return salary


<>:13: SyntaxWarning: invalid escape sequence '\d'
<>:13: SyntaxWarning: invalid escape sequence '\d'
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_3828/1908726828.py:13: SyntaxWarning: invalid escape sequence '\d'
  end_year = max(re.findall('\d{4,}', end_contract))


In [24]:
df['salaire'] = df['ID'].apply(salaire)
df['salaire']

391     64 000 €
2626    31 000 €
1964     8 000 €
479     13 000 €
495     60 000 €
Name: salaire, dtype: object